In [89]:
#| hide
#| eval:false
! [ -e /content ] && pip install -Uqq fastai # upgrade fastai on colab

In [90]:
#| default_exp l2r.callbacks

In [91]:
#| export
from fastai.torch_imports import *
from fastai.torch_core import *
from fastai.callback.core import *
from fastcore.all import *
from xcube.imports import *
from xcube.metrics import *

In [92]:
#| hide
from nbdev.showdoc import *

In [93]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Callbacks for L2R

> General purpose callbacks needed for L2R learner

In [94]:
#| export
class TrackResults(Callback):
    def __init__(self, train_metrics=False): 
        store_attr()
        self.names = ['loss', 'ndcg', 'ndcg_at_6', 'acc']
    
    def before_fit(self): self.losses_full, self.grads_full, self.metrics_full = [], defaultdict(list), defaultdict(list) 
    
    def before_epoch(self): self.losses, self.ndcgs, self.ndcgs_at_6, self.accs = [], [], [], []
    
    def after_epoch(self):
        if self.model.training: self.losses_full.extend(self.losses)
        _li = [self.losses, self.ndcgs, self.ndcgs_at_6, self.accs]
        _li = [torch.stack(o) if o else torch.Tensor() for o in _li] 
        [self.losses, self.ndcgs, self.ndcgs_at_6, self.accs] = _li
        log = [round(o.mean().item(), 4) if o.sum() else "NA" for o in _li]
        if self.model.training:
            if self.train_metrics: self.metrics_full['trn'].append(log)
        else: self.metrics_full['val'].append(log)
        print(self.epoch, self.model.training, *log)
    
    def after_batch(self):
        with torch.no_grad():
            loss = self.loss_func(self.preds, self.xb)
            self.losses.append(loss.mean())
            if self.model.training:
                if self.train_metrics: self._compute_metrics()
            else: self._compute_metrics()
                        
    def _compute_metrics(self):
        *_, _ndcg, _ndcg_at_k = ndcg(self.preds, self.xb, k=6)
        self.ndcgs.append(_ndcg.mean())
        self.ndcgs_at_6.append(_ndcg_at_k.mean())
        acc = accuracy(self.xb, self.model).mean()
        self.accs.append(acc.mean())
        
    def after_backward(self):
        for name,param in self.model.named_parameters():
            grad = param.grad.data.detach().clone()
            self.grads_full[name].append(grad)

In [95]:
#| export
class ProgressBarCallback(Callback):
    order = 70
    
    def before_fit(self):
        self.mbar = master_bar(range(self.n_epochs))
        
    def before_epoch(self):
        if getattr(self, 'mbar', False): self.mbar.update(self.epoch)
        self._launch_pbar()
        
    def _launch_pbar(self):
        self.pbar = progress_bar(self.dl, parent=getattr(self, 'mbar', None), leave=False)
        self.pbar.update(0)
        
    def after_batch(self):
        self.pbar.update(self.iter_num+1)
    
    def after_epoch(self):
        self.pbar.on_iter_end()
        
    def after_fit(self):
        if getattr(self, 'mbar', False):
            self.mbar.on_iter_end()
            delattr(self, 'mbar')

In [96]:
#| export
class Monitor(Callback):
    order = 60
    
    def __init__(self, monitor='ndcg_at_6', comp=None, min_delta=0., reset_on_fit=False):
        if comp is None: comp = np.greater
        if comp == np.less: min_delta *= -1
        store_attr()
        self.best = None
       
    def before_fit(self):
        if self.reset_on_fit or self.best is None: self.best = float('inf') if self.comp == np.less else -float('inf')
        assert self.monitor in self.track_results.names
        self.idx = list(self.track_results.names).index(self.monitor)
        
    def after_epoch(self):
        val = self.track_results.metrics_full.get('val')[-1][self.idx]
        if self.comp(val - self.min_delta, self.best): self.best, self.new_best, = val, True
        else: self.new_best = False
    

In [97]:
#| export
class SaveCallBack(Monitor):
    order = Monitor.order+1
    def __init__(self, fname, monitor='ndcg_at_6', comp=None, min_delta=0., reset_on_fit=False):
        super().__init__(monitor=monitor, comp=comp, min_delta=min_delta, reset_on_fit=reset_on_fit)
        self.last_saved_path = None
        store_attr()
        
    def after_epoch(self):
        super().after_epoch()
        if self.new_best:
            print(f'Better model found at epoch {self.epoch} with {self.monitor} value: {self.best}.')
            self.learn.save(self.fname)

## Export

In [98]:
#| hide
import nbdev; nbdev.nbdev_export()